# Data loading and preproccessing
First of, we will laod the data and take the select few articles related to data science and computers, with one outlier being Memory article.

In [1]:
import json

target_articles = {}
# Loading articles from dev
with open('../../data/stanford-question-answering-dataset/dev-v1.1.json', 'r', encoding='utf-8') as f:
    squad_data_dev = json.load(f)
for i, article in enumerate(squad_data_dev['data']):
    if i==4 or i==19 :
        target_articles[article['title']] = article['paragraphs']
# Loading articles from train
with open('../../data/stanford-question-answering-dataset/train-v1.1.json', 'r', encoding='utf-8') as f2:
    squad_data_train = json.load(f2)
for i, article in enumerate(squad_data_train['data']):
    if i in {30, 62, 104, 124, 149, 157, 177, 229, 293, 295, 330, 333, 389, 393} :
        target_articles[article['title']] = article['paragraphs']

In [2]:
for article_title, paragraphs in target_articles.items():
    print(f"""Article: {article_title}""")

Article: Computational_complexity_theory
Article: Packet_switching
Article: Wayback_Machine
Article: Web_browser
Article: Computer
Article: Computer_security
Article: ASCII
Article: Macintosh
Article: Memory
Article: Data_compression
Article: USB
Article: Unicode
Article: IBM
Article: Software_testing
Article: Database
Article: Printed_circuit_board


After loading in wanted the articles, we can prepare text for chunking and questions for testing the retrieval.

In [3]:
from langchain_core.documents import Document

docs_for_splitter = []
questions_ground_truth = []
p_idx=0
for article_title, paragraphs in target_articles.items():
    for _, para_data in enumerate(paragraphs):
        context_text = para_data['context']
        
        # Creating documents with needed metadata
        doc = Document(
            page_content=context_text,
            metadata={
                "para_id": p_idx,
                "article": article_title
            }
        )
        docs_for_splitter.append(doc)
        
        # Extracting questions and linking them to doc that should be retrieved for them
        for qa in para_data['qas']:
            questions_ground_truth.append({
                "question": qa['question'],
                "ground_truth_para_id": p_idx,
                # "qa_id": qa['id'],
                # "article": article_title
            })
        p_idx+=1

Finally, we will save this data for later use in pkl format.

In [ ]:
import pickle

output_path = "../../data/processed/squad_processed.pkl"
output_data = {
    "docs_for_splitter": docs_for_splitter,
    "questions_ground_truth": questions_ground_truth
}

with open(output_path, "xb") as f:
    pickle.dump(output_data, f)

print(f"Data successfully exported to {output_path}")

Data successfully exported to ../../data/processed/squad_processed.pkl
